In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# downloading training data from MNIST
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

# download test data from open datasets
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

100%|██████████| 26421880/26421880 [00:30<00:00, 856498.54it/s] 


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 139040.54it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3284081.21it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 10548254.51it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [3]:
# to enable batching we pass batch_size to the DataLoader
batch_size=64

# DataSet & DataLoader object type:
# DataSet contains samples and corresponding labels
# DataLoader wraps an iterable over the DataSet

# create dataloaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of vector of images X [N, C, H, W]: {X.shape}")
    print(f"Shape of labels y: {y.shape} {y.dtype}")
    break

Shape of image X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of vector y: torch.Size([64]) torch.int64


In [4]:
# Create a model: get cpu, gpu or mps device for training
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [6]:
# define the NN model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512), # takes input img of 28x28 converts to 512 dim
            nn.ReLU(), # activation func for weights
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)  # takes the 512 output of prev layer and outputs 10 classes
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [8]:
# optimizing model params
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

# training loop (makes preds on training dataset in batches)
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader): # number of images to labels
        # pass the image and vectors to device
        X, y = X.to(device), y.to(device)
        
        # compute the pred err
        pred = model(X)
        loss = loss_fn(pred, y) # loss between the prediction and the labels
        
        # backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
        

In [10]:
# check the model's performance against the test dataset
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    print("dataset size = {}", size)
    print("num_batches = {}", num_batches)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader: # number of images to labels
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [12]:
# training process over several epochs. during each epoch, the model learns
# parameters to make better predictions. Print model's acc and loss at each
# epoch; accuracy increase and loss decrease with every epoch
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------
loss: 1.137284 [   64/60000]
loss: 1.138809 [ 6464/60000]
loss: 0.953371 [12864/60000]
loss: 1.098211 [19264/60000]
loss: 0.975974 [25664/60000]
loss: 0.987837 [32064/60000]
loss: 1.040583 [38464/60000]
loss: 0.975051 [44864/60000]
loss: 1.019662 [51264/60000]
loss: 0.958803 [57664/60000]
dataset size = {} 10000
num_batches = {} 157
Test error: 
 Accuracy: 65.5%, Avg loss: 0.969079 

Epoch 2
-------------------------
loss: 1.026928 [   64/60000]
loss: 1.049960 [ 6464/60000]
loss: 0.847328 [12864/60000]
loss: 1.013678 [19264/60000]
loss: 0.894199 [25664/60000]
loss: 0.899015 [32064/60000]
loss: 0.969574 [38464/60000]
loss: 0.908553 [44864/60000]
loss: 0.945601 [51264/60000]
loss: 0.898746 [57664/60000]
dataset size = {} 10000
num_batches = {} 157
Test error: 
 Accuracy: 66.9%, Avg loss: 0.901893 

Epoch 3
-------------------------
loss: 0.944699 [   64/60000]
loss: 0.987599 [ 6464/60000]
loss: 0.770192 [12864/60000]
loss: 0.952876 [19264/60000]
loss: 0.

KeyboardInterrupt: 

In [14]:
# Saving the models
torch.save(model.state_dict(), "model.pth")
print("Saved the Torch model state to model.pth")

Saved the Torch model state to model.pth


In [15]:
# Load the models
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [25]:
# make predictions using test classes
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
